<a href="https://colab.research.google.com/github/KalpanaMehta/KalpanaMehta/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pwd

/content


In [8]:
!mkdir deep_learning

In [9]:
%cd deep_learning/

/content/deep_learning


In [11]:
!mkdir images

In [12]:
%cd images

/content/deep_learning/images


In [13]:
!mkdir colorization

In [14]:
!mkdir colorization2

In [15]:
cd colorization/

/content/deep_learning/images/colorization


In [37]:
# !mkdir class1
# !mkdir class2
%cd /content/deep_learning/images/

/content/deep_learning/images


In [38]:
from google.colab import files
upload = files.upload()

Saving tranfer_Image.jpeg to tranfer_Image.jpeg


In [39]:
from keras.applications import ResNet50
model = ResNet50()

In [40]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [41]:
from keras.preprocessing.image import load_img
image = load_img('/content/deep_learning/images/tranfer_Image.jpeg',target_size=(224,224))


In [42]:
#converting pixels to numpy array

from keras.preprocessing.image import img_to_array
image = img_to_array(image)

In [43]:
import numpy as np
image = np.expand_dims(image,axis =0)

In [44]:
from keras.applications.vgg16 import preprocess_input
image = preprocess_input(image)

In [45]:
pred = model.predict(image)

1/1 [==============================] - 2s 2s/step


In [46]:
from tensorflow.keras.applications.mobilenet import decode_predictions
pred_classes = decode_predictions(pred, top=5)
for i in pred_classes[0]:
    print(i)

35363/35363 [==============================] - 0s 0us/step
('n01614925', 'bald_eagle', 0.93017083)
('n01795545', 'black_grouse', 0.031570375)
('n01798484', 'prairie_chicken', 0.010568842)
('n02002724', 'black_stork', 0.008273316)
('n01608432', 'kite', 0.0068677864)


In [47]:
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb,gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
import os

In [48]:
from keras.applications.vgg16 import VGG16
vgg_model = VGG16()

553467096/553467096 [==============================] - 15s 0us/step


In [49]:
newmodel = Sequential()
for i, layer in enumerate(vgg_model.layers):
    if i<19:          #Only up to 19th layer to include feature extraction only
      newmodel.add(layer)
newmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [50]:
for layer in newmodel.layers:
  layer.trainable=False

In [51]:
path = '/content/deep_learning/images/colorization'
#Normalize images - divide by 255
training_data = ImageDataGenerator(rescale=1. / 255)


In [52]:
train = training_data.flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode=None)

Found 4 images belonging to 2 classes.


In [53]:
datagen = ImageDataGenerator(rescale=1./255)

# Use flow_from_directory to create a generator
generator = datagen.flow_from_directory(
    path,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None
)

Found 4 images belonging to 2 classes.


In [54]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128,
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(4, 224, 224, 1)
(4, 224, 224, 2)


In [55]:
vggfeatures = []
for i, sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = newmodel.predict(sample)
  prediction = prediction.reshape((7,7,512))
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

1/1 [==============================] - 1s 529ms/step
(4, 7, 7, 512)


In [56]:
#Decoder
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 256)         1179904   
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         295040    
                                                                 
 up_sampling2d (UpSampling2  (None, 14, 14, 128)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 up_sampling2d_1 (UpSamplin  (None, 28, 28, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)       

In [57]:

model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=10, batch_size=128)

model.save('colorize_autoencoder_VGG16.model')

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6683 - accuracy: 0.1647
Epoch 2/10
1/1 [==============================] - 0s 206ms/step - loss: 0.8522 - accuracy: 0.7132
Epoch 3/10
1/1 [==============================] - 0s 250ms/step - loss: 0.8333 - accuracy: 0.8575
Epoch 4/10
1/1 [==============================] - 0s 188ms/step - loss: 0.7285 - accuracy: 0.7944
Epoch 5/10
1/1 [==============================] - 0s 164ms/step - loss: 0.5128 - accuracy: 0.2630
Epoch 6/10
1/1 [==============================] - 0s 133ms/step - loss: 0.0767 - accuracy: 0.6796
Epoch 7/10
1/1 [==============================] - 0s 137ms/step - loss: 0.0336 - accuracy: 0.7600
Epoch 8/10
1/1 [==============================] - 0s 143ms/step - loss: 0.0158 - accuracy: 0.7824
Epoch 9/10
1/1 [==============================] - 0s 138ms/step - loss: 0.0139 - accuracy: 0.7994
Epoch 10/10
1/1 [==============================] - 0s 136ms/step - loss: 0.0121 - accuracy: 0.8093


In [58]:
model = tf.keras.models.load_model('colorize_autoencoder_VGG16.model',
                                   custom_objects=None,
                                   compile=True)


In [67]:
testpath = '/images/colorization2/eagle9.jpeg'
files = os.listdir(testpath)
for idx, file in enumerate(files):
    test = img_to_array(load_img(testpath+file))
    test = resize(test, (224,224), anti_aliasing=True)
    test*= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    ab = ab*128
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    imsave('images/colorization2/vgg_result/result'+str(idx)+".jpg", lab2rgb(cur))

FileNotFoundError: ignored

In [64]:
!pwd

/content/deep_learning/images
